# Configuração do Ambiente AdventureWorks2025

In [0]:
%python    

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import re
import time
import uuid
from datetime import datetime

In [0]:
--drop catalog if exists adventureworks cascade;
CREATE CATALOG IF NOT EXISTS adventureworks;
USE CATALOG adventureworks;

CREATE SCHEMA IF NOT EXISTS bronze;
CREATE VOLUME IF NOT EXISTS bronze.delta_tables;

CREATE SCHEMA IF NOT EXISTS silver;
CREATE VOLUME IF NOT EXISTS silver.delta_tables;

CREATE SCHEMA IF NOT EXISTS gold;
CREATE VOLUME IF NOT EXISTS gold.delta_tables;

CREATE SCHEMA IF NOT EXISTS metadata;



In [0]:
%python
# Configurar caminhos no Workspace
base_path = '/Volumes/adventureworks'
bronze_path = f"{base_path}/bronze/delta_tables"
silver_path = f"{base_path}/silver/delta_tables"
gold_path = f"{base_path}/gold/delta_tables"


# Criar diretórios
dbutils.fs.mkdirs(bronze_path)
dbutils.fs.mkdirs(silver_path)
dbutils.fs.mkdirs(gold_path)


In [0]:
CREATE TABLE IF NOT EXISTS metadata.etl_control (
  table_name STRING,
  layer STRING,
  last_processed_date TIMESTAMP,
  records_count BIGINT,
  status STRING,
  error_message STRING,
  processing_time DOUBLE,
  created_at TIMESTAMP
)
USING DELTA;

In [0]:
%python
def log_etl(table_name, layer, status, records=0, error=None, duration=0):
    """Registra execução do ETL"""
    spark.sql(f"""
        INSERT INTO metadata.etl_control 
        VALUES (
            '{table_name}',
            '{layer}',
            CURRENT_TIMESTAMP(),
            {records},
            '{status}',
            {f"'{error}'" if error else 'NULL'},
            {duration},
            CURRENT_TIMESTAMP()
        )
    """)
    print(f"{'SUCCESS' if status=='SUCCESS' else 'ERROR'} {layer}.{table_name}: {status} - {records} registros - {duration}s")





In [0]:
%python
def to_snake_case(name: str) -> str:
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower()